In [3]:
import pennylane as qml
import numpy as np
import random

In [261]:
dev = qml.device("default.mixed", wires=3)
@qml.qnode(dev)
def generate_noise(wires):
    density_matrix = generate_density_matrix()
    qml.QubitDensityMatrix(density_matrix, wires=wires)
    # qml.BitFlip(0.9,wires[2])
    return qml.density_matrix(wires)

In [262]:
def uniformly_sample_pair(S, L):
  ret = []
  for i in range(L):
    ret.append(random.sample(S, 2))
  return ret

In [263]:
def generate_density_matrix():
    # this is the hadamard shit
    c1 = 1/np.sqrt(2)
    c2 = 1/np.sqrt(2)
    state = np.array([0.5, 0.5, 0.5, 0,0,0,0.5,0])
    
    # generate the density matrix
    rho = np.outer(state, np.conj(state).T)
    return rho

In [264]:
G1 = qml.Identity(0) @ qml.PauliZ(1) @ qml.PauliZ(2)
G2 = qml.PauliZ(0) @ qml.PauliZ(1) @ qml.Identity(2)

G1 = G1.matrix()
G2 = G2.matrix()

I = qml.Identity(0) @ qml.Identity(1) @ qml.Identity(2)
I = I.matrix()

O = qml.Identity(0) @ qml.Identity(1) @ qml.PauliZ(2)
O = O.matrix()

p1 = 0.5 * (I + G1)
p2 = 0.5 * (I + G2)
P = p1 @ p2

rho = generate_noise([0,1,2])
rho_det = (P @ rho @ P) / (np.trace(rho @ P))

In [271]:

def generate_fig_2(U_list, O):
    n_estimation_wires = len(U_list)
    n_target_wires = 3

    dev = qml.device("default.mixed", wires=(n_estimation_wires + n_target_wires), shots=1)

    @qml.qnode(dev)
    def VQED(rho_naught, stabilizers_list, estimation_wire, target_wires):
        measurements = []
        qml.QubitDensityMatrix(rho_naught, wires=target_wires)
        for wire in estimation_wires:
            qml.Hadamard(wires=wire)

        for l, (stabilizer_i_l, stabilizer_j_l) in enumerate(stabilizers_list):
            U = U_list[l]
            qml.QubitUnitary(U, wires=target_wires)
            qml.ControlledQubitUnitary(stabilizer_i_l, control_wires=estimation_wires[l], wires=target_wires, control_values=[False])
            qml.ControlledQubitUnitary(stabilizer_j_l, control_wires=estimation_wires[l], wires=target_wires, control_values=[True])
        
        L = len(stabilizers_list)

        for wire in estimation_wires:
            measurements.append(qml.expval(
                qml.PauliX(wire)
            ))

        measurements.append(qml.expval(qml.Hermitian(O, wires=target_wires)))
        return measurements

    return VQED

U1 = qml.Identity(0) @ qml.Identity(1) @ qml.Identity(2)
U1 = U1.matrix()
U2 = qml.Identity(0) @ qml.Identity(1) @ qml.Identity(2)
U2 = U2.matrix()
U3 = qml.Identity(0) @ qml.Identity(1) @ qml.Identity(2)
U3 = U3.matrix()

Ulist = [U1, U2, U3]

f = generate_fig_2(Ulist, O)

# S size of 4 (Assuming generators that we chose should be able to make up the set of S)
S = [G1, G2, G1 @ G2, I]
# L = 4

estimation_wires=[0, 1, 2]
target_wires=[3,4,5]

a = 0
b = 0
N = 1000


for i in range(N):
    uniform_sample = uniformly_sample_pair(S, len(estimation_wires))
    measurements = f(rho,uniform_sample,estimation_wires,target_wires)
    a_s = measurements[0]
    for sample in measurements[1:-1]:
        a_s *= sample
    b_s = a_s * measurements[-1]
    a += a_s
    b += b_s

a *= 1/N
b *= 1/N
print(f"VQED b/a: {b/a}")
tr = np.trace(rho_det @ O)
print(f"tr[p_detO]: {tr}")
err = abs((tr - b / a) / tr) * 100
print(f"err: {err}%")



VQED b/a: -0.4
tr[p_detO]: (1+0j)
err: 140.0%


In [94]:
# symmetry expansion

ans = np.trace(rho_det @ O)
print(f"tr[p_detO]: {ans}")

num = np.trace(rho @ O @ P)
denom = np.trace(rho @ P)
print(f"tp[pOP]/tr[pP]: {num/denom}")

N = 10000

S = [G1, G2, G1 @ G2, I]
a = 0
b = 0

for i in range(N):
    Si = random.sample(S,1)[0]
    a_s = np.trace(rho @ Si)
    b_s = np.trace(rho @ O @ Si)
    a += a_s
    b += b_s


print(f"Symmettry Expansion b/a:{b / a}")



tr[p_detO]: (0.05263157894737286+0j)
tp[pOP]/tr[pP]: (0.05263157894737283+0j)
Symmettry Expansion b/a:(0.056692413501218765+0j)
